In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from influxdb import *
# import Config as cg
from influxdb import DataFrameClient
from datetime import datetime, timedelta
import pytz


class ENERGY24():

#     def __init__(self):
#         self.DFDBClient = DataFrameClient(host=cg.INFLUX_DB_IP, port=cg.INFLUX_DB_PORT, database=cg.INFLUX_DB)

    def __call__(self):
        self.output()

    def difference(self, df1):
        diff = []
        for g, i in df1.groupby(pd.Grouper(freq='H', key='time'))['EM_TOTAL_Import_Energy(kWh)']:
            try:
                diff.append([g, i.iloc[-1] - i.iloc[0]])
            except:
                diff.append([g, np.nan])
        x = pd.DataFrame(diff)
        return x

    def one_hr_agg(self, df1):
        o = df1.groupby('DeviceID').apply(self.difference)
        o = o.pivot_table(index=0, columns='DeviceID', values=1).reset_index()
        o.columns.name = None
        o = o.rename(columns={0: ""})
        o = o.set_index("")
        o = o.fillna(0)
        return o

    def read_Data(self):
        INFLUX_DB_IP = '10.12.97.178'
        INFLUX_DB_PORT = 8086
        INFLUX_DB = 'demoDb'
        TARGET_MEASUREMENT = 'EM_main_new'
        con_obj = InfluxDBClient(host=INFLUX_DB_IP, port=INFLUX_DB_PORT, database=INFLUX_DB)
        query = 'select "EM_TOTAL_Import_Energy(kWh)", "DeviceID", "time" from ' + TARGET_MEASUREMENT + ' where time > now() - 150d '
        df1 = pd.DataFrame(con_obj.query(query).get_points())
        # df1 = pd.DataFrame(con_obj.query(query).get_points())
        df1['time'] = df1['time'].astype('datetime64[ns]')
        df1['time'] = df1['time'] + timedelta(hours=5, minutes=30)
        df4 = self.one_hr_agg(df1)
        Time_range = pd.DataFrame(pd.date_range(start=(df4.index.min()), end=(df4.index.max()), freq='H'))
        Time_range = Time_range.rename(columns={0: "time"})
        Time_range.set_index("time", inplace=True)
        df = df4.merge(Time_range, how="outer", right_index=True, left_index=True)
        df = df.interpolate(method='time')
        df = df.fillna(0)
        df.index.freq = 'H'
        return df

    def output(self):
        try:
            df = self.read_Data()
            return df
        except Exception as e:
            print(e)


if __name__ == '__main__':
    cat = ENERGY24()
    t = cat.output()

In [4]:
# t[t<0]=0

In [2]:
t

,EM1,EM10,EM2,EM22,EM23,EM3,EM4,EM5,EM6,EM7,EM8,EM9
2020-05-29 19:00:00,0.0,0.00,0.0,0.00,0.00,0.0,13.0,26.0,0.00,0.0,0.0,0.0
2020-05-29 20:00:00,0.0,0.00,0.0,0.00,0.00,0.0,24.0,47.0,0.00,0.0,0.0,0.0
2020-05-29 21:00:00,0.0,0.00,0.0,0.00,0.00,0.0,24.0,39.0,0.00,0.0,0.0,0.0
2020-05-29 22:00:00,79.0,0.00,0.0,0.00,0.00,0.0,24.0,40.0,0.00,0.0,0.0,0.0
2020-05-29 23:00:00,139.0,0.00,0.0,0.00,0.00,0.0,24.0,40.0,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-04 09:00:00,905.0,37.33,0.0,3.64,24.57,0.0,76.0,176.0,4.55,0.1,0.0,40.4
2020-09-04 10:00:00,831.0,37.04,0.0,3.63,24.58,0.0,36.0,171.0,1.21,0.1,0.0,17.4
2020-09-04 11:00:00,842.0,37.39,0.0,3.64,24.59,0.0,39.0,172.0,1.16,0.0,0.0,20.6
2020-09-04 12:00:00,847.0,37.24,0.0,3.63,24.56,0.0,38.0,172.0,1.40,0.1,0.0,18.8


In [4]:
df=t[['EM4']]

In [6]:
df.index.min()

Timestamp('2020-05-29 19:00:00', freq='H')

In [7]:
df.index.max()

Timestamp('2020-09-04 13:00:00', freq='H')

In [9]:
df.to_excel("data_2020-05-29_to_2020-09-04.xlsx")